In [4]:
import pandas as pd
import numpy as np
from numpy import log, dot, exp, shape
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score

# Load data
data = pd.read_csv('C://Users//Admin//Downloads//archive (4)//suv_data.csv')
print(data.head())

# Features and target
x = data.iloc[:, [2, 3]].values  # Age, EstimatedSalary
y = data.iloc[:, 4].values       # Purchased

# -------------------- In-built Logistic Regression --------------------
# Split and scale
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.10, random_state=0)
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

# Train
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state=0)
classifier.fit(x_train, y_train)

# Predict and evaluate
y_pred = classifier.predict(x_test)
print("Predictions:", y_pred)

cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)

print("Accuracy:", accuracy_score(y_test, y_pred))

# -------------------- User-defined Logistic Regression --------------------
# Split again (if needed)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.10, random_state=0)

# Standardization helper
def standardize(X_tr):
    for i in range(shape(X_tr)[1]):
        X_tr[:, i] = (X_tr[:, i] - np.mean(X_tr[:, i])) / np.std(X_tr[:, i])

# F1-score function
def F1_score(y, y_hat):
    tp = tn = fp = fn = 0
    for i in range(len(y)):
        if y[i] == 1 and y_hat[i] == 1:
            tp += 1
        elif y[i] == 1 and y_hat[i] == 0:
            fn += 1
        elif y[i] == 0 and y_hat[i] == 1:
            fp += 1
        elif y[i] == 0 and y_hat[i] == 0:
            tn += 1
    precision = tp / (tp + fp) if (tp + fp) != 0 else 0
    recall = tp / (tp + fn) if (tp + fn) != 0 else 0
    f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
    return f1_score

# Logistic Regression class
class LogisticRegressionCustom:
    def sigmoid(self, z):
        return 1 / (1 + exp(-z))

    def initialize(self, X):
        weights = np.zeros((shape(X)[1] + 1, 1))
        X = np.c_[np.ones((shape(X)[0], 1)), X]
        return weights, X

    def fit(self, X, y, alpha=0.001, iter=400):
        weights, X = self.initialize(X)
        y = y.reshape(-1, 1)

        def cost(theta):
            z = dot(X, theta)
            cost0 = y.T.dot(log(self.sigmoid(z)))
            cost1 = (1 - y).T.dot(log(1 - self.sigmoid(z)))
            return -((cost1 + cost0)) / len(y)

        cost_list = np.zeros(iter)
        for i in range(iter):
            z = dot(X, weights)
            h = self.sigmoid(z)
            weights -= alpha * dot(X.T, h - y)
            cost_list[i] = cost(weights)
        self.weights = weights
        return cost_list

    def predict(self, X):
        _, X = self.initialize(X)
        z = dot(X, self.weights)
        preds = self.sigmoid(z)
        return [1 if i > 0.5 else 0 for i in preds]

# Apply standardization
standardize(x_train)
standardize(x_test)

# Train and predict
obj = LogisticRegressionCustom()
model = obj.fit(x_train, y_train)
y_pred_custom = obj.predict(x_test)
y_train_pred = obj.predict(x_train)

# Evaluate
f1_score_train = F1_score(y_train, y_train_pred)
f1_score_test = F1_score(y_test, y_pred_custom)

print("F1 Score (Train):", f1_score_train)
print("F1 Score (Test):", f1_score_test)

    User ID  Gender  Age  EstimatedSalary  Purchased
0  15624510    Male   19            19000          0
1  15810944    Male   35            20000          0
2  15668575  Female   26            43000          0
3  15603246  Female   27            57000          0
4  15804002    Male   19            76000          0
Predictions: [0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0
 0 0 1]
Confusion Matrix:
 [[31  1]
 [ 1  7]]
Accuracy: 0.95
F1 Score (Train): 0.7583333333333334
F1 Score (Test): 0.823529411764706


C:\Users\Admin\AppData\Local\Temp\ipykernel_2608\452243154.py:88: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cost_list[i] = cost(weights)
